## Imports

In [1]:
%cd ..

d:\Kerjaan\Kuliah\Tugas Akhir


In [2]:
import pandas as pd
import requests

## Data Exploration

<h4> Several findings </h4>
<li> Dalam pemakaian data, cukup digunakan train_2 dan key_2 saja
<li> NaN dan 0 tidak dibedakan, jadi arti dari NaN mungkin karena memang webpagenya belum ada atau karena memang 0 views

In [3]:
train_1 = pd.read_csv('./data/web-traffic-time-series-forecasting/train_1.csv')
train_2 = pd.read_csv('./data/web-traffic-time-series-forecasting/train_2.csv')
key_1 = pd.read_csv('./data/web-traffic-time-series-forecasting/key_1.csv')
key_2 = pd.read_csv('./data/web-traffic-time-series-forecasting/key_2.csv')

In [16]:
print(len(train_1))
train_1.head(5)

145063


,Page,2015-07-01,2015-07-02,2015-07-03,2015-07-04,2015-07-05,2015-07-06,2015-07-07,2015-07-08,2015-07-09,...,2016-12-22,2016-12-23,2016-12-24,2016-12-25,2016-12-26,2016-12-27,2016-12-28,2016-12-29,2016-12-30,2016-12-31
0,2NE1_zh.wikipedia.org_all-access_spider,18.0,11.0,5.0,13.0,14.0,9.0,9.0,22.0,26.0,...,32.0,63.0,15.0,26.0,14.0,20.0,22.0,19.0,18.0,20.0
1,2PM_zh.wikipedia.org_all-access_spider,11.0,14.0,15.0,18.0,11.0,13.0,22.0,11.0,10.0,...,17.0,42.0,28.0,15.0,9.0,30.0,52.0,45.0,26.0,20.0
2,3C_zh.wikipedia.org_all-access_spider,1.0,0.0,1.0,1.0,0.0,4.0,0.0,3.0,4.0,...,3.0,1.0,1.0,7.0,4.0,4.0,6.0,3.0,4.0,17.0
3,4minute_zh.wikipedia.org_all-access_spider,35.0,13.0,10.0,94.0,4.0,26.0,14.0,9.0,11.0,...,32.0,10.0,26.0,27.0,16.0,11.0,17.0,19.0,10.0,11.0
4,52_Hz_I_Love_You_zh.wikipedia.org_all-access_s...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,48.0,9.0,25.0,13.0,3.0,11.0,27.0,13.0,36.0,10.0


In [5]:
print(len(train_2))
train_2.head(3)

145063


,Page,2015-07-01,2015-07-02,2015-07-03,2015-07-04,2015-07-05,2015-07-06,2015-07-07,2015-07-08,2015-07-09,...,2017-09-01,2017-09-02,2017-09-03,2017-09-04,2017-09-05,2017-09-06,2017-09-07,2017-09-08,2017-09-09,2017-09-10
0,2NE1_zh.wikipedia.org_all-access_spider,18.0,11.0,5.0,13.0,14.0,9.0,9.0,22.0,26.0,...,19.0,33.0,33.0,18.0,16.0,27.0,29.0,23.0,54.0,38.0
1,2PM_zh.wikipedia.org_all-access_spider,11.0,14.0,15.0,18.0,11.0,13.0,22.0,11.0,10.0,...,32.0,30.0,11.0,19.0,54.0,25.0,26.0,23.0,13.0,81.0
2,3C_zh.wikipedia.org_all-access_spider,1.0,0.0,1.0,1.0,0.0,4.0,0.0,3.0,4.0,...,6.0,6.0,7.0,2.0,4.0,7.0,3.0,4.0,7.0,6.0


In [6]:
print(key_1.shape)
key_1.head()

(8703780, 2)


,Page,Id
0,!vote_en.wikipedia.org_all-access_all-agents_2...,bf4edcf969af
1,!vote_en.wikipedia.org_all-access_all-agents_2...,929ed2bf52b9
2,!vote_en.wikipedia.org_all-access_all-agents_2...,ff29d0f51d5c
3,!vote_en.wikipedia.org_all-access_all-agents_2...,e98873359be6
4,!vote_en.wikipedia.org_all-access_all-agents_2...,fa012434263a


In [7]:
print(key_2.shape)
key_2.head()

(8993906, 2)


,Page,Id
0,007_スペクター_ja.wikipedia.org_all-access_all-agen...,0b293039387a
1,007_スペクター_ja.wikipedia.org_all-access_all-agen...,7114389dd824
2,007_スペクター_ja.wikipedia.org_all-access_all-agen...,057b02ff1f09
3,007_スペクター_ja.wikipedia.org_all-access_all-agen...,bd2aca21caa3
4,007_スペクター_ja.wikipedia.org_all-access_all-agen...,c0effb42cdd5


## Data Preprocessing

### Data Scraping

In [49]:
def get_wiki_parameters(parameter_str):
    parameter_list = parameter_str.split('_')
    agent = parameter_list[-1]
    access = parameter_list[-2]
    project = parameter_list[-3]
    name = '_'.join(parameter_list[:-3])

    return {
        'path': parameter_str,
        'name':name,
        'project': project,
        'access': access,
        'agent': agent
    }

In [21]:
pd.DataFrame.from_dict(train_2.iloc[:5].set_index('Page').to_dict(orient='index'))

,2NE1_zh.wikipedia.org_all-access_spider,2PM_zh.wikipedia.org_all-access_spider,3C_zh.wikipedia.org_all-access_spider,4minute_zh.wikipedia.org_all-access_spider,52_Hz_I_Love_You_zh.wikipedia.org_all-access_spider
2015-07-01,18.0,11.0,1.0,35.0,NaN
2015-07-02,11.0,14.0,0.0,13.0,NaN
2015-07-03,5.0,15.0,1.0,10.0,NaN
2015-07-04,13.0,18.0,1.0,94.0,NaN
2015-07-05,14.0,11.0,0.0,4.0,NaN
...,...,...,...,...,...
2017-09-06,27.0,25.0,7.0,16.0,23.0
2017-09-07,29.0,26.0,3.0,19.0,28.0
2017-09-08,23.0,23.0,4.0,30.0,14.0
2017-09-09,54.0,13.0,7.0,38.0,8.0


In [50]:
wiki_parameters = [get_wiki_parameters(parameter_str) for parameter_str in train_2['Page'].tolist()]

In [36]:
for parameter in wiki_parameters:
    project = parameter['project']
    access = parameter['access']
    agent = parameter['agent']
    article = parameter['name']
    granularity = "daily"
    start = '20170910'
    end = '20171231'
    response = requests.get(
        f'https://wikimedia.org/api/rest_v1/metrics/pageviews/per-article/{project}/{access}/{agent}/{article}/{granularity}/{start}/{end}', 
        headers={'User-Agent': 'belmiro533backup@gmail.com'}
    )
    break

print(response.json())

{'items': [{'project': 'zh.wikipedia', 'article': '2NE1', 'granularity': 'daily', 'timestamp': '2015070100', 'access': 'all-access', 'agent': 'spider', 'views': 18}, {'project': 'zh.wikipedia', 'article': '2NE1', 'granularity': 'daily', 'timestamp': '2015070200', 'access': 'all-access', 'agent': 'spider', 'views': 11}]}
